In [1]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import warnings

warnings.filterwarnings(action='ignore') 
know_train = [pd.read_csv(path) for path in sorted(glob('./train/*.csv'))]

In [2]:
know_train[0].head()

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3,knowcode
0,0,3,3,3,3,3,3,4,4,3,...,52,2,실업,1,1,1,4000,,2200,825101
1,1,4,5,4,5,3,4,3,4,3,...,38,4,건축공학,1,1,1,,,2400,140204
2,2,3,4,3,4,3,4,5,6,4,...,50,4,건축공학,1,1,1,4000,,2400,140204
3,3,3,3,3,3,3,5,4,5,4,...,42,4,환경학과,1,1,1,7000,,3500,140601
4,4,4,5,3,4,3,4,4,5,3,...,51,4,건축공학,1,1,1,4000,,2500,140204


In [3]:
for df in know_train:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [4]:
from sklearn.preprocessing import LabelEncoder
years = ['2017', '2018', '2019', '2020']

year_encoder = {}

for year, df in zip(years, know_train):
    print(year)
    encoders = {}
    
    for col in df.columns:
        if col == 'ID':
            continue
        
        try:
            df[col] = df[col].map(int)
        except:
            encoder = LabelEncoder()
            df[col] = df[col].map(str)
            df[col] = encoder.fit_transform(df[col])
            encoders[col] = encoder
            
            
    year_encoder[year] = encoders

2017
2018
2019
2020


In [5]:
train_data = {}
for year, df in zip(years, know_train):
    train_data[year] = {'X': df.iloc[:, 1:-1], # ID제외
                        'y': df.iloc[:, -1]} 

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

dt_models = {}

for year in tqdm(years):
    model = DecisionTreeClassifier(random_state=123456)
    model.fit(train_data[year]['X'].iloc[:, :], train_data[year]['y'])
    dt_models[year] = model

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  2.30s/it]


In [7]:
rf_models = {}

for year in tqdm(years):
    model = RandomForestClassifier(n_estimators=100, random_state=123456, n_jobs=8)
    model.fit(train_data[year]['X'].iloc[:, :], train_data[year]['y'])
    rf_models[year] = model

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:26<00:00,  6.72s/it]


In [8]:
know_test = [pd.read_csv(path) for path in sorted(glob('./test/*.csv'))]
know_test[0].head() # 2017년도 test 샘플

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3
0,0,3,4,2,2,3,3,1,,3,...,2,26,3,비서학,1,1,1,3000,,2300
1,1,5,5,3,5,5,5,5,5,4,...,1,57,4,농화학,1,1,1,5500,,2500
2,2,5,5,5,4,5,4,1,,1,...,1,31,4,신문방송,1,1,1,4300,,4000
3,3,4,5,5,6,4,6,3,4,4,...,1,35,6,화학,1,1,1,4100,,3000
4,4,5,6,4,5,4,5,1,,1,...,1,36,4,광고홍보,1,1,1,2800,,2000


In [9]:
for df in know_test:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [10]:
years = ['2017', '2018', '2019', '2020']

for year, df in zip(years, know_test):
    print(year)
    encoders = {}
    
    for col in df.columns:
        
        try:
            df[col] = df[col].map(int)
        except:
            encoder = year_encoder[year][col]
            df[col] = df[col].map(str)
            category_map = {category: idx for idx, category in enumerate(encoder.classes_)}
            df[col] = df[col].apply(lambda x: category_map[x] if x in category_map else -1) # train set에서 보지못한 카테고리변수 -1(UNK) 처리
            

2017
2018
2019
2020


In [11]:
know_train[0]['aq1_1'].value_counts()

3    3116
4    3007
2    1706
5    1072
1     585
Name: aq1_1, dtype: int64

In [12]:
test_data = {}
for year, df in zip(years, know_test):
    test_data[year] =  {'X': df.iloc[:,1:]}

In [13]:
dt_predicts = [] 

for year in tqdm(years):
    pred = dt_models[year].predict(test_data[year]['X'])
    dt_predicts.extend(pred)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 11.28it/s]


In [14]:
rf_predicts = [] 

for year in tqdm(years):
    pred = rf_models[year].predict(test_data[year]['X'])
    rf_predicts.extend(pred)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:14<00:00,  3.57s/it]


In [15]:
submission = pd.read_csv('sample_submission.csv') # sample submission 불러오기

In [16]:
submission['knowcode'] = rf_predicts

submission.to_csv('submission.csv', index=False)